# LangChain으로 웹에서 정보 수집하기 - URL

- LangChain을 이용하여 빠르게 변하는 IT 소식, 영어로 되어있는 **TechCrunch 뉴스 기사**를 자동으로 번역 및 요약하고, 궁금한 내용에 대해 바로 답변을 받아보는 URL QA 봇을 만들어봅시다.

## 필요한 모듈을 설치합니다.

In [1]:
!pip3 install langchain==0.0.268 openai==0.28.1 tiktoken youtube-transcript-api pytube finance-datareader google-search-results googletrans==3.1.0a0

### 자료 출처: "The week in AI: Generative AI spams up the web" TechCrunch 뉴스 기사
- 링크: https://techcrunch.com/2023/07/08/the-week-in-ai-generative-ai-spams-up-the-web/

## 발급 받은 OpenAI key, Google-Search-API Key를 불러옵니다.

In [2]:
import os

os.environ["OPENAI_API_KEY"] = "sk-iqikeXmsMIaJo73WNYPtQARmKCdhr-IUY4yNJjgWxJT3BlbkFJSjjKw6pNAyeNKntKTicdtpx6Sv4It1Cm_4_yZ6E2oA"
os.environ["SERPAPI_API_KEY"] = "30cb94d750ef7aed4a81c3c9481ee2fa12272ebba8cf772039642bfecf269af0"

## URL 내의 텍스트 데이터를 불러옵니다.
- URL 링크를 2개 이상 작성하면 여러 개의 링크로부터 텍스트 데이터를 모두 가져올 수 있습니다.

In [3]:
!pip install unstructured

  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached httpx-0.27.2-py3-none-any.whl.metadata (7.1 kB)
  Using cached idna-3.8-py3-none-any.whl.metadata (9.9 kB)
  Using cached httpcore-1.0.5-py3-none-any.whl.metadata (20 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
Using cached dataclasses_json-0.6.7-py3-none-any.whl (28 kB)
Using cached httpx-0.27.2-py3-none-any.whl (76 kB)
Using cached httpcore-1.0.5-py3-none-any.whl (77 kB)
Using cached idna-3.8-py3-none-any.whl (66 kB)
Using cached h11-0.14.0-py3-none-any.whl (58 kB)
  Attempting uninstall: idna
    Found existing installation: idna 2.10
    Uninstalling idna-2.10:
      Successfully uninstalled idna-2.10
  Attempting uninstall: h11
    Found existing installation: h11 0.9.0
    Uninstalling h11-0.9.0:
      Successfully uninstalled h11-0.9.0
  Attempting uninstall: httpcore
    Found existing installation: httpcore 0.9.1
    Uninstalling httpcore-0.9.1:
      Successfully uninst

In [4]:
!pip install nltk

In [5]:
import nltk
nltk.data.path.append('/root/nltk_data/tokenizers/punkt/PY3_tab')
nltk.download('punkt', download_dir='/root/nltk_data/tokenizers/punkt/PY3_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     /root/nltk_data/tokenizers/punkt/PY3_tab...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
from langchain.document_loaders import UnstructuredURLLoader

nltk.download('punkt')

urls = ["https://techcrunch.com/2023/07/08/the-week-in-ai-generative-ai-spams-up-the-web/",
        "https://techcrunch.com/2024/09/01/the-case-against-ai-art/"]

loader = UnstructuredURLLoader(urls=urls)
data = loader.load()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [7]:
news_raw_text = ""

for el in data:
    news_raw_text += (el.page_content + " ")

## 영어 뉴스를 우선 한글로 번역해봅니다.
- 이전 실습에서 사용했던 translator 함수를 그대로 사용합니다.

In [8]:
!pip install --upgrade httpcore httpx

In [1]:
!pip install --upgrade googletrans==4.0.0-rc1

In [2]:
import googletrans

def translator(sent, lang='ko'):
    translator = googletrans.Translator()
    result = translator.translate(sent, dest=lang)
    return result.text

In [3]:
# 번역할 텍스트 정의
news_raw_text = "Artificial intelligence is rapidly transforming industries around the world."


In [4]:
korean_news_raw_text = translator(news_raw_text)

## 이제 뉴스에서 중요한 내용을 요약해봅니다.
- 이전 실습에서 사용했던 qa_bot 함수를 그대로 사용합니다.

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import AnalyzeDocumentChain
from langchain.callbacks import get_openai_callback
import FinanceDataReader as fdr
import time

def qa_bot(source, question, model='gpt-4o-mini', temperature=0, chain_type="map_reduce"):

    exchange_rate = fdr.DataReader('USD/KRW').iloc[-1][0]

    model = ChatOpenAI(model=model, temperature=0)

    qa_chain = load_qa_chain(model, chain_type=chain_type)
    qa_document_chain = AnalyzeDocumentChain(combine_docs_chain=qa_chain)

    with get_openai_callback() as cb:

        start = time.time()
        print(qa_document_chain.run(input_document=source, question=question))
        end = time.time()

        print(f"Total Tokens: {cb.total_tokens}")
        print(f"Prompt Tokens: {cb.prompt_tokens}")
        print(f"Completion Tokens: {cb.completion_tokens}")
        print(f"Total Cost (USD): {cb.total_cost:.2f}$, Total Cost (Won): {cb.total_cost*exchange_rate:.2f}₩")
        print(f"걸린 시간: {end-start:.2f}초")

### Option 1: 영어 뉴스를 바로 한글로 요약합니다.

In [8]:
import os

os.environ['OPENAI_API_KEY'] = 'sk-iqikeXmsMIaJo73WNYPtQARmKCdhr-IUY4yNJjgWxJT3BlbkFJSjjKw6pNAyeNKntKTicdtpx6Sv4It1Cm_4_yZ6E2oA'

In [9]:
qa_bot(news_raw_text, "이 글을 한글로 요약해줘.")

<ipython-input-7-fc60052bdb6c>:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  exchange_rate = fdr.DataReader('USD/KRW').iloc[-1][0]


인공지능이 전 세계 산업을 빠르게 변화시키고 있다는 내용입니다.
Total Tokens: 178
Prompt Tokens: 146
Completion Tokens: 32
Total Cost (USD): 0.00$, Total Cost (Won): 0.00₩
걸린 시간: 3.77초


### Option 2: 번역된 한글 뉴스에 대해 요약합니다.

In [11]:
qa_bot(korean_news_raw_text, "이 글을 보고서 형식으로 요약해줘.")

<ipython-input-7-fc60052bdb6c>:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  exchange_rate = fdr.DataReader('USD/KRW').iloc[-1][0]


보고서 제목: 인공 지능의 산업 변화

1. 서론
   - 본 보고서는 인공 지능(AI)의 발전이 전 세계 산업에 미치는 영향에 대해 논의한다.

2. 본론
   - 인공 지능은 전 세계 산업을 빠르게 변화시키고 있으며, 다양한 분야에서의 응용이 증가하고 있다.

3. 결론
   - 인공 지능의 지속적인 발전은 산업 구조와 운영 방식에 중대한 변화를 가져올 것으로 예상된다.
Total Tokens: 443
Prompt Tokens: 237
Completion Tokens: 206
Total Cost (USD): 0.00$, Total Cost (Won): 0.00₩
걸린 시간: 2.86초


## 뉴스를 보다가 모르는 것이 있으면 바로 질문할 수 있는 나만의 백과사전 만들어봅시다.
- 뉴스를 이해해다가 모르는 것이 있으면 구글 검색 API를 LangChain에 연결하여 뉴스 이해를 도와주는 나만의 검색 엔진 Agent를 만들 수 있습니다.
- 무료 계정의 경우 총 100회까지만 이용할 수 있음에 주의합니다.

In [14]:
import os

os.environ["SERPAPI_API_KEY"] = "30cb94d750ef7aed4a81c3c9481ee2fa12272ebba8cf772039642bfecf269af0"

In [15]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI


llm = ChatOpenAI(temperature=0, model='gpt-4o-mini')

tool_names = ["serpapi"]
tools = load_tools(tool_names)

agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [16]:
agent.run("SpeedyBrand 회사에 대해 좀더 소개해줄래?")



> Entering new AgentExecutor chain...
SpeedyBrand에 대한 정보를 찾기 위해 검색해보아야겠다. 
Action: Search
Action Input: "SpeedyBrand 회사 소개" 
Observation: SpeedyBrand is revolutionizing content marketing with AI for businesses. Create high-quality, personalized, SEO-optimized blogs and social media content in ...
Thought:SpeedyBrand는 AI를 활용하여 기업의 콘텐츠 마케팅을 혁신하고 있으며, 고품질의 개인화된 SEO 최적화 블로그 및 소셜 미디어 콘텐츠를 생성하는 서비스를 제공하는 회사이다. 
Final Answer: SpeedyBrand는 AI를 활용하여 기업의 콘텐츠 마케팅을 혁신하고, 고품질의 개인화된 SEO 최적화 블로그 및 소셜 미디어 콘텐츠를 생성하는 서비스를 제공하는 회사이다.

> Finished chain.


'SpeedyBrand는 AI를 활용하여 기업의 콘텐츠 마케팅을 혁신하고, 고품질의 개인화된 SEO 최적화 블로그 및 소셜 미디어 콘텐츠를 생성하는 서비스를 제공하는 회사이다.'

In [17]:
agent.run("Y Combinator가 뭔지 알려줘.")



> Entering new AgentExecutor chain...
Y Combinator는 스타트업을 지원하는 액셀러레이터 프로그램으로, 초기 단계의 기업에 자금을 투자하고 멘토링을 제공하는 기관입니다. 이 프로그램은 스타트업이 성장할 수 있도록 돕기 위해 다양한 리소스를 제공합니다. 더 구체적인 정보가 필요할 수 있으므로, Y Combinator의 최근 활동이나 프로그램에 대한 업데이트를 확인해보겠습니다. 
Action: Search
Action Input: Y Combinator 최근 활동 및 프로그램 정보
Observation: We help founders at their earliest stages regardless of their age. We improve the success rate of our startups. We give startups a huge fundraising advantage.
Thought:Y Combinator는 초기 단계의 스타트업을 지원하는 액셀러레이터 프로그램으로, 자금 지원과 멘토링을 통해 스타트업의 성공률을 높이고 있습니다. 또한, 스타트업이 자금을 모으는 데 유리한 조건을 제공합니다. 최근에도 이러한 지원을 지속하고 있다는 것을 알 수 있습니다. 
Final Answer: Y Combinator는 초기 단계의 스타트업에 자금을 투자하고 멘토링을 제공하는 액셀러레이터 프로그램으로, 스타트업의 성공률을 높이고 자금 조달에 유리한 조건을 제공합니다.

> Finished chain.


'Y Combinator는 초기 단계의 스타트업에 자금을 투자하고 멘토링을 제공하는 액셀러레이터 프로그램으로, 스타트업의 성공률을 높이고 자금 조달에 유리한 조건을 제공합니다.'

In [20]:
# 번역하기
translator('We help founders at their earliest stages regardless of their age. We improve the success rate of our startups. We give startups a huge fundraising advantage.')

'우리는 나이에 관계없이 초기 단계의 창립자를 도와줍니다.우리는 스타트 업의 성공률을 향상시킵니다.우리는 신생 기업에 큰 기금 모금 이점을 제공합니다.'

## 이전 챕터에서 배운 질문 답변 뉴스 QA 봇을 활용하여 뉴스를 입체적으로 이해해봅시다.

### Q1. 이 글의 주제가 뭐니?

In [21]:
qa_bot(korean_news_raw_text, "이 글을 요약해줘.")

<ipython-input-7-fc60052bdb6c>:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  exchange_rate = fdr.DataReader('USD/KRW').iloc[-1][0]


인공 지능이 전 세계 산업을 빠르게 변화시키고 있다는 내용입니다.
Total Tokens: 181
Prompt Tokens: 147
Completion Tokens: 34
Total Cost (USD): 0.00$, Total Cost (Won): 0.00₩
걸린 시간: 1.26초


### Q2. 글에서 openai와 GPT에 대한 내용을 찾아줄래?

In [22]:
qa_bot(korean_news_raw_text, "글에서 openai와 GPT에 대한 내용을 찾아줄래?")

<ipython-input-7-fc60052bdb6c>:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  exchange_rate = fdr.DataReader('USD/KRW').iloc[-1][0]


제공된 텍스트에는 OpenAI와 GPT에 대한 내용이 포함되어 있지 않습니다.
Total Tokens: 204
Prompt Tokens: 162
Completion Tokens: 42
Total Cost (USD): 0.00$, Total Cost (Won): 0.00₩
걸린 시간: 1.19초


### Q3. 이 글에서 SpeedyBrand 말고 다른 회사들은 어떤 뉴스가 있었는지 차례대로 정리해줘.

In [23]:
qa_bot(korean_news_raw_text, "이 글에서 SpeedyBrand 말고 다른 회사들은 어떤 뉴스가 있었는지 차례대로 정리해줘")

<ipython-input-7-fc60052bdb6c>:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  exchange_rate = fdr.DataReader('USD/KRW').iloc[-1][0]


제공된 텍스트에는 SpeedyBrand이나 다른 회사들에 대한 뉴스에 대한 정보가 포함되어 있지 않습니다. 추가적인 정보가 필요합니다.
Total Tokens: 299
Prompt Tokens: 214
Completion Tokens: 85
Total Cost (USD): 0.00$, Total Cost (Won): 0.00₩
걸린 시간: 1.44초


### Q4. 최근에 openai에서 무언가를 개발하고 있다는 내용이 있어?

In [24]:
qa_bot(korean_news_raw_text, "최근에 openai에서 무언가를 개발하고 있다는 내용이 있어?")

<ipython-input-7-fc60052bdb6c>:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  exchange_rate = fdr.DataReader('USD/KRW').iloc[-1][0]


제공된 텍스트에는 OpenAI에서 최근에 개발하고 있다는 내용이 포함되어 있지 않습니다.
Total Tokens: 216
Prompt Tokens: 170
Completion Tokens: 46
Total Cost (USD): 0.00$, Total Cost (Won): 0.00₩
걸린 시간: 1.26초


### Q5. 해당 글에서 영화 Elemental 제작에 대한 이야기가 있던데 어떤 내용인지 소개해줄래?

In [25]:
qa_bot(korean_news_raw_text, "해당 글에서 영화 Elemental 제작에 대한 이야기가 있던데 어떤 내용인지 소개해줄래?")

<ipython-input-7-fc60052bdb6c>:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  exchange_rate = fdr.DataReader('USD/KRW').iloc[-1][0]


해당 글에는 영화 Elemental 제작에 대한 내용이 포함되어 있지 않습니다.
Total Tokens: 255
Prompt Tokens: 199
Completion Tokens: 56
Total Cost (USD): 0.00$, Total Cost (Won): 0.00₩
걸린 시간: 1.48초


### Q6. 글에서 SpeedyBrand 회사의 투자에 대한 내용이 있었어?

In [26]:
qa_bot(korean_news_raw_text, "글에서 SpeedyBrand 회사의 투자에 대한 내용이 있었어?")

<ipython-input-7-fc60052bdb6c>:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  exchange_rate = fdr.DataReader('USD/KRW').iloc[-1][0]


제공된 텍스트에는 SpeedyBrand 회사의 투자에 대한 내용이 포함되어 있지 않습니다.
Total Tokens: 214
Prompt Tokens: 168
Completion Tokens: 46
Total Cost (USD): 0.00$, Total Cost (Won): 0.00₩
걸린 시간: 0.97초


### Q7. 글에서 게임과 관련된 내용에 대해 소개해줄래?

In [27]:
qa_bot(korean_news_raw_text, "글에서 게임과 관련된 내용에 대해 소개해줄래?")

<ipython-input-7-fc60052bdb6c>:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  exchange_rate = fdr.DataReader('USD/KRW').iloc[-1][0]


제공된 텍스트에는 게임과 관련된 내용이 포함되어 있지 않습니다. 인공 지능이 전 세계 산업을 변화시키고 있다는 내용만 있습니다.
Total Tokens: 251
Prompt Tokens: 179
Completion Tokens: 72
Total Cost (USD): 0.00$, Total Cost (Won): 0.00₩
걸린 시간: 1.37초


### Q8. 딥페이크 관련한 이야기가 보이던데 딥페이크가 무엇인지 알려주고 무슨 내용이 있었는지 알려줘

In [28]:
qa_bot(korean_news_raw_text, "딥페이크 관련한 이야기가 보이던데 딥페이크가 무엇인지 알려주고 무슨 내용이 있었는지 알려줘")

<ipython-input-7-fc60052bdb6c>:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  exchange_rate = fdr.DataReader('USD/KRW').iloc[-1][0]


딥페이크는 인공지능 기술을 사용하여 사람의 얼굴이나 목소리를 합성하여 가짜 비디오나 오디오를 만드는 기술을 의미합니다. 제공된 텍스트에는 딥페이크에 대한 구체적인 내용이나 사례는 포함되어 있지 않습니다. 따라서 딥페이크와 관련된 더 많은 정보는 제공된 텍스트에서 찾을 수 없습니다.
Total Tokens: 419
Prompt Tokens: 256
Completion Tokens: 163
Total Cost (USD): 0.00$, Total Cost (Won): 0.00₩
걸린 시간: 3.01초


## [TODO] qa_bot 함수를 활용해서, 웹문서 기반으로 QA를 수행해 보고 웹 사전도 활용해보세요.

In [52]:
!pip install deep-translator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.6 MB/s eta 0:00:00


In [53]:
# 웹문서 url 입력하여 텍스트 가져오기
from langchain.document_loaders import UnstructuredURLLoader
from deep_translator import GoogleTranslator

# 웹문서 url 입력하여 텍스트 가져오기
urls = [
    "https://www.samsungsds.com/kr/news/sds-genai-mediaday.html",
    "https://www.newswire.co.kr/newsRead.php?no=989024"
]

loader = UnstructuredURLLoader(urls=urls)
data = loader.load()


In [54]:
# 텍스트 전처리
raw_text = "AI"

for el in data:
    raw_text += (el.page_content + " ")

In [58]:
# (필요 시) 번역하기
# 번역하기 - deep-translator를 사용하여 번역
translator = GoogleTranslator(source='auto', target='ko')


In [59]:
# 웹문서 기반으로 질문해보기
question = "Brity Copilot 대해 설명해"
qa_bot(raw_text, question)

<ipython-input-7-fc60052bdb6c>:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  exchange_rate = fdr.DataReader('USD/KRW').iloc[-1][0]


Brity Copilot은 삼성SDS가 제공하는 협업 솔루션으로, 회사 업무에서 자주 사용하는 메일, 메신저, 미팅, 문서 관리 등의 공통 업무를 지원합니다. 이 서비스는 생성형 AI 기술을 적용하여 다양한 업무를 효율화할 수 있도록 돕습니다. 

Brity Copilot의 주요 기능은 다음과 같습니다:
- 영상회의 중 발표자의 음성을 인식해 실시간 자막(한/영) 및 번역(13개 언어 가능)
- 회의 전체 내용의 자막 스크립트 제공
- 회의록 작성 및 실행 방안 도출
- 메일/메신저 대화 요약
- 메일 초안 작성

특히, 한국어 음성의 높은 인식 정확도(94%)와 실시간 번역 기능으로 전문 통역사 없이도 명확한 커뮤니케이션이 가능하여, 글로벌 비즈니스를 하는 기업에 유용합니다. 또한, Brity Copilot은 기업이 원하는 다양한 LLM(거대언어모델)과 연계할 수 있으며, 보안이 필요한 고객을 위한 프라이빗 클라우드 환경도 지원합니다. 삼성SDS는 Brity Copilot을 통해 회의록 작성 시간을 75% 이상, 메일 작성 시 내용 요약 및 초안 작성에 걸리던 시간을 66% 이상 절감할 수 있다고 밝혔습니다.
Total Tokens: 5030
Prompt Tokens: 4107
Completion Tokens: 923
Total Cost (USD): 0.00$, Total Cost (Won): 0.00₩
걸린 시간: 10.64초


In [60]:
# 모르는 단어에 대해 에이전트를 활용하여 웹서칭 해보기
agent.run("Hyper-automation")



> Entering new AgentExecutor chain...
Hyper-automation is a term that refers to the use of advanced technologies, such as artificial intelligence (AI), machine learning (ML), and robotic process automation (RPA), to automate complex business processes and functions. It goes beyond traditional automation by integrating multiple automation tools and technologies to create a more comprehensive and efficient automation strategy. 

To provide a more detailed and current understanding of hyper-automation, I will search for recent developments or insights related to this topic. 

Action: Search  
Action Input: "hyper-automation 2023 trends and developments"  
Observation: In 2023, we can expect an increase in the adoption of low-code/no-code tools, as they empower business users to automate tasks, processes, and ...
Thought:I have gathered information about the current trends and developments in hyper-automation for 2023. It appears that there is a significant focus on low-code/no-code tool

'Hyper-automation in 2023 is characterized by the rise of low-code/no-code tools that enable business users to automate processes easily, along with a greater integration of AI and machine learning to enhance decision-making and efficiency in complex business processes.'